In [1]:
import requests as r
import pandas as pd

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import metabaseapi_utils as mb
sys.path.pop()

import os
import dotenv

dotenv.load_dotenv()
mb_name = os.environ["MS_METABASE_NAME"]
mb_pw = os.environ["MS_METABASE_PW"]

mb_url_base = "https://dash.goldsky.com"

# https://goldsky.notion.site/SHARED-Lightweight-API-Documentation-for-Goldsky-Dashboarding-5cde15ba222844f485c31a4426f6ed53

In [2]:
# Map Chain Names
chain_mappings = {
    'zora': 'Zora',
    'pgn': 'Public Goods Network',
    # Add more mappings as needed
}

In [3]:
session_id = mb.get_session_id(mb_url_base, mb_name, mb_pw)

# try: #do you already have a session
#         session_id = os.environ["MS_METABASE_SESSION_ID"]
# except: #if not, make one
#         print('creating new session')
#         session_id = mb.get_mb_session_key(mb_url_base,mb_name,mb_pw)


<Response [401]>
An error occurred: 401 Client Error: Unauthorized for url: https://dash.goldsky.com/api/card/42/query/json
Maximum number of retries (1) reached due to exception. Giving up.
creating new session


In [1]:
# print(session_id)
if (os.environ["IS_RUNNING_LOCAL"]):
        print(session_id)

NameError: name 'os' is not defined

In [5]:
# Run Query

#21-op-chains-activity-by-day
query_num = 21
print(query_num)
resp = mb.get_mb_query_response(mb_url_base, session_id, query_num, num_retries = 3)
# print(resp)

21
<Response [200]>
[{"num_raw_txs": 78872, "num_blocks": 43200, "l2_num_txs_per_day": 35017, "l2_gas_used": 9410861821, "l1_gas_paid": 84642569.58422089, "l2_gas_fees_priority_fee": 1.956766179243754, "dt": "2023-11-09", "base_fee_gwei": 5.0000265465978486e-08, "l2_eth
{'num_raw_txs': 78872, 'num_blocks': 43200, 'l2_num_txs_per_day': 35017, 'l2_gas_used': 9410861821, 'l1_gas_paid': 84642569.58422089, 'l2_gas_fees_priority_fee': 1.956766179243754, 'dt': '2023-11-09', 'base_fee_gwei': 5.0000265465978486e-08, 'l2_eth_fees_per_day': 5.684810068558113, 'avg_l1_gas_price_on_l2': 44.0445454367355, 'num_users_per_day': 20126, 'l2_gas_used_user_txs_per_day': 7227713287, 'num_user_txs_per_second': 0.40528935185185183, 'active_secs_per_day': 86400, 'l2_num_attr_deposit_txs_per_day': 43200, 'l1_gas_used_user_txs_l2_per_day': 123746444, 'l2_gas_used_per_block': 217844.02363425927, 'chain_type': 'mainnet', 'l2_num_user_deposit_txs_per_day': 1, 'l1_gas_paid_user_txs': 84642569.58422089, 'calldata_ga

In [6]:
data_df = pd.DataFrame(resp)

data_df['chain'] = data_df['chain'].replace(chain_mappings)
# data_df['dt'] = pd.to_datetime(data_df['dt'])

print(data_df.columns)
# print(data_df.dtypes)

data_df = data_df.sort_values(by='dt',ascending=False)

print(data_df.head(5))

Index(['num_raw_txs', 'num_blocks', 'l2_num_txs_per_day', 'l2_gas_used',
       'l1_gas_paid', 'l2_gas_fees_priority_fee', 'dt', 'base_fee_gwei',
       'l2_eth_fees_per_day', 'avg_l1_gas_price_on_l2', 'num_users_per_day',
       'l2_gas_used_user_txs_per_day', 'num_user_txs_per_second',
       'active_secs_per_day', 'l2_num_attr_deposit_txs_per_day',
       'l1_gas_used_user_txs_l2_per_day', 'l2_gas_used_per_block',
       'chain_type', 'l2_num_user_deposit_txs_per_day', 'l1_gas_paid_user_txs',
       'calldata_gas_user_txs_l2_per_day', 'avg_l2_gas_price',
       'calldata_bytes_user_txs_l2_per_day', 'is_initial_day',
       'l1_contrib_l2_eth_fees_per_day', 'calldata_bytes_l2_per_day',
       'l2_gas_used_per_second', 'equivalent_l1_tx_fee',
       'l2_contrib_l2_eth_fees_per_day', 'calldata_gas_l2_per_day',
       'l1_gas_used_on_l2', 'l2_gas_fees_base_fee', 'l2_eth_fees_per_second',
       'l2_eth_fees_per_block', 'chain', 'l2_num_txs_per_day_per_block'],
      dtype='object')
   n

In [7]:
# Post to Dune API
d.write_dune_api_from_pandas(data_df, 'opchain_activity_by_day_gs',\
                             'Basic Daily Activity for OP Chains - Zora & PGN (from Goldsky)')

table at: dune.oplabspbc.dataset_opchain_activity_by_day_gs
Response status code: 200
Response content: b'{"success":true,"table_name":"opchain_activity_by_day_gs"}'
